In [35]:
from pathlib import Path

from time import time

import copy

import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.optim import lr_scheduler, SGD
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [36]:
def prepare_data(path: Path):
    # Data transformation need for ResNet18. It applies only basic cropping
    # and normalization.
    data_transforms = {
        "train":
            transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
            ]),
        "val":
            transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
            ])
    }

    # Creates dataset based on a given path.
    image_datasets = {
        mode: ImageFolder(path / mode, data_transforms[mode])
        for mode in ["train", "val"]
    }
    #Creates dataloaders from ImageFolders.
    dataloaders = {
        mode: DataLoader(image_datasets[mode],
                         batch_size=4,
                         shuffle=True,
                         num_workers=4) for mode in ["train", "val"]
    }

    dataset_sizes = {
        mode: len(image_datasets[mode]) for mode in ["train", "val"]
    }
    class_names = image_datasets["train"].classes
    return dataloaders, dataset_sizes, class_names

In [37]:
def train_model(dataloaders,
                model,
                criterion,
                optimizer,
                scheduler,
                num_epochs=15,
                device="cpu"):
    start = time()

    # Need to keep best model.
    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Repeat traning num_epochs time.
    for epoch in range(num_epochs):
        print(20 * "-")
        print(f"Epoch {epoch}/{num_epochs - 1}")

        # Each iteration consists of a training phase and a validation phase.
        for mode in ["train", "val"]:
            if mode == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[mode]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(mode == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Perform backpropagation only in training mode.
                    if mode == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if mode == "train":
                scheduler.step()

            # Calculate loss and accuracy.
            epoch_loss = running_loss / dataset_sizes[mode]
            epoch_acc = running_corrects.double() / dataset_sizes[mode]

            print(f"{mode} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # Save model if it is the best so far.
            if mode == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model.state_dict())
        print()

    end = time()
    time_elapsed = end - start
    print(
        f"Training complete in {time_elapsed // 60:.0f}min {time_elapsed % 60:.0f}s"
    )
    print(f"Best val Acc: {best_acc:4f}")

    model.load_state_dict(best_model)
    return model

In [38]:
def create_model(device="cpu"):
    # Download pretrained model.
    model = models.resnet18(pretrained=True)

    # Freeze ResNet18 disabling requires_grad.
    for param in model.parameters():
        param.requires_grad = False

    # Newly constructed module has requires_grad=True by default.
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 3)

    model = model.to(device)

    return model

In [39]:
dataloaders, dataset_sizes, class_names = prepare_data(
    Path("../data/sharks_subset_tests"))

model = create_model()

criterion = nn.CrossEntropyLoss()
# Only parameters of final layer are being optimized.
optimizer = SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [40]:
train_model(dataloaders, model, criterion, optimizer, scheduler);

--------------------
Epoch 0/14
train Loss: 1.0175 Acc: 0.5049
val Loss: 1.4237 Acc: 0.4273

--------------------
Epoch 1/14
train Loss: 0.8084 Acc: 0.6324
val Loss: 0.5947 Acc: 0.7091

--------------------
Epoch 2/14
train Loss: 0.6662 Acc: 0.6814
val Loss: 0.4379 Acc: 0.7818

--------------------
Epoch 3/14
train Loss: 0.6164 Acc: 0.7598
val Loss: 0.3931 Acc: 0.8364

--------------------
Epoch 4/14
train Loss: 0.7434 Acc: 0.6912
val Loss: 0.4757 Acc: 0.8364

--------------------
Epoch 5/14
train Loss: 0.6746 Acc: 0.7206
val Loss: 0.3098 Acc: 0.8545

--------------------
Epoch 6/14
train Loss: 0.7386 Acc: 0.6961
val Loss: 0.5183 Acc: 0.8000

--------------------
Epoch 7/14
train Loss: 0.5781 Acc: 0.7010
val Loss: 0.3572 Acc: 0.8455

--------------------
Epoch 8/14
train Loss: 0.4691 Acc: 0.8039
val Loss: 0.3995 Acc: 0.8364

--------------------
Epoch 9/14
train Loss: 0.4770 Acc: 0.8186
val Loss: 0.3941 Acc: 0.8273

--------------------
Epoch 10/14
train Loss: 0.5285 Acc: 0.7941
val Lo